Alternative: brwn.co/br3

### Frequentist statistics ###

The axioms of probability have been worked out for centuries. If you've had a statistics class you've seen them somewhere in the first third of the class. 

1. The probability of an event is greater than or equal to 0
2. The probability that at least one of all the possible events that might occur in an experiment is 1
3. If two events A and B are mutually exclusive, then the probability of either occurring is the sum of the probability of A and the probability of B.

These were formalized by Andrey Kolmogorov in 1933. From these three you can derive an incredible number of consequences including the so-called Law of Averages first derived by Jacob Bernoulli. It says that if you have a "trial" (think of a coin toss) with some event that occurs with probability p (think the coin lands heads and p=0.5), then the proportion of times your event occurs in n trials tends to p. That's a lot. Here's a picture of, say, tossing a fair coin repeatedly and looking at the relative frequency of heads among the outcomes.

<img src=https://github.com/computationaljournalism/columbia2020/raw/master/stats-breakfasts/o.jpg>

Notice that the more often we toss a fair coin, the proportion of heads settles down to the probability of 1/2. That this happens is a mathematical consequence of the axioms of probability. 

In his “A Treatise on Probability,” the British Economist John Maynard Keynes discusses several attempts to verify the conclusions of Bernoulli’s Theorem — he writes “I record them because they have a good deal of historical and psychological interest, and because they satisfy a certain idle curiosity from which few students of probability are altogether free.”

>The French naturalist Count Buffon (1707-1788), who “assisted by a child tossing a coin into the air” recorded 2048 heads in 4040 flips (for a relative frequency of 0.507)
<br><br>
A similar experiment was carried out by a student of the British mathematician De Morgan (1806-1871) “for his own satisfaction” involving 4092 tosses, 2048 of which were heads (relative frequency of 0.500
<br><br>
The Belgian mathematician/astronomer/statistician/sociologist Adolphe Quetelet (1796-1874) drew 4096 balls from an urn, replacing them each time, and recorded the result at different stages; in all, he drew 2066 white balls and 2030 black balls (relative frequency of 0.504)
<br><br>
English economist W S Jevons (1835-1882) made 2048 throws of ten coins at a time; in all, he saw 20,480 tosses out of which 10,353 were heads (relative frequency of 0.506, although this is not quite the same kind of trial)
<br><br>Around 1900, the English statistician Karl Pearson (1857-1936) made two heroic studies; the first involved 12,000 tosses (relative frequency of heads 0.52) and the second 24,000 times (12,012 of which landed heads for a relative frequency of 0.501)
<br><br>While imprisoned by the Germans during World War II, the South African mathematician John Kerrich tossed a coin 10,000 times, 5067 of them heads (this gives a relative frequency of 0.5067 — while interned, he also recorded a monograph “An Experimental Introduction to the Theory of Probability”

This connection between relative frequencies and probability leads to one interpretation of probability. The axioms give us a way to compute things mathematically, but they don't really provide us with an interpretation of what probability *means.* So-called frequentists, then, would interpret the probability of a coin landing heads as being the limit approached by the relative frequency of seeing heads as you toss the coin some large number of times.

You can tell when you are dealing with frequentist statistics because there is always an imagined set of trials in the background somewhere. For Arbuthnot, the experiment was tossing 82 coins to see how many came up heads. For Hill, it was re-randomizing patients into treatment and control. 

In both cases, the inferential procedures we have studied (re-randomization and P-values) are based on this frequentist notion of probability — they refer to an (imaginary) set of possible alternative outcomes that could have happened had we repeated an experiment many times. 

Well-known statistician D.R. Cox puts it this way

>In the first so-called frequentist approach, we ... use probability as representing a long-run frequency... [W]e measure uncertainty via procedures such as confidence limits and significance levels (P-values), whose behaviour ... is assessed by considering hypothetically how they perform when used repeatedly under the same conditions. The performance may be studied analytically or by computer simulation.
<br><br>
In that, the procedure is calibrated by what happens when it is used, it is no different from other measuring devices.

Today we are going to look at another frequentist construction, the confidence interval.

### Confidence Intervals ###

I had thought I would begin with lessons from COVID-19 but will leave that to next time. We are in the midst of the 2020 Census and I'll take today's lesson from there. 

And so we are going to start with a case that would never happen in practice, but is good for teaching. Using the tables in [Summary File 1 or SF1](https://www.socialexplorer.com/data/C2000/metadata/?ds=SF1) from the 2010 census that we  [downloaded from the Census Bureau](https://www.census.gov/data/datasets/2010/dec/summary-file-1.html), we applied  the so-called ["database reconstruction theorem" to reconstruct (with remarkable accuracy) the person-level responses to 2010 Census](https://www.nytimes.com/2018/12/05/upshot/to-reduce-privacy-risks-the-census-plans-to-report-less-accurate-data.html) questions about sex, age, ethnicity, and race. 

Wait, what? The Census Bureau discovered (and we repeated) that you could take all the aggregate statistics they publish (7 billion in all for 2010) and reconstruct the person-level table from which they were derived. This realization prompted a whole new program of disclosure avoidance to be adopted for 2020, one that will have big implications for anyone using the data at a block level. 

Below we will load in the person-level tables for all of Manhattan as reconstructed in this way. This give us a table with 1,585,873 rows, one for each person living in Manhattan. As these data were a census, they were extremely expensive to obtain, and we only undergo this process every 10 years. This gives us an opportunity to talk about statistical ideas like sampling and confidence intervals and the bootstrap.

The census data will serve as the population we are trying to learn something about. In this case, we happen to have all the answers from everyone in the population. It will cost us nothing to compute the "true" population parameters we might be interested in. We can also use the `sample_n()` command in R to conduct as many snap surveys of the population as we want... and the data are real, so there's no extra layer of distracting simulation here. 

First, let's read in the census data. Again, this can be considered actual person-level data from everyone living in Manhattan in 2010 (the reconstruction is exactly right for 50% of the population and 90% contains just one mistake, almost always age and almost the age is off by a year or two -- we will ignore these errors and take these data to be the answers from the population living in Manhattan).

To load the data, you need to download it from [this Dropbox link](https://www.dropbox.com/s/mxsukoowq2ds8cf/manhattan_census.csv?dl=0). Place the file `manhattan_census.csv` into the same folder as this notebook is in. Then you should be able to read it with the code below.

In [ ]:
census <- read.csv("manhattan_census.csv")
head(census)

And sure enough there are 1,585,873 rows, each row a person's response. I've also included the tract people live in. Tracts are made up of blocks, the census' finest level of geography, but I decided not to include block for privacy reasons. It's actually possible to use the block-level reconstruction to correctly identify about 20% of the population.

In [ ]:
dim(census)

We are going to use the entire city as our population, but we could use these data to ask about differences between different neighborhoods. The tracts I provided are enough to create New York City's Neighborhood Tabulation Areas. They are [given here](https://data.cityofnewyork.us/City-Government/2010-Census-Tract-to-Neighborhood-Tabulation-Area-/8ius-dhrr). You can see a map of the names of the neighborhoods referenced in this table [here](https://www1.nyc.gov/assets/planning/download/pdf/data-maps/nyc-population/census2010/ntas.pdf)

Let's have a look just to keep things real. 

In [ ]:
nbhds <- read.csv("https://data.cityofnewyork.us/api/views/8ius-dhrr/rows.csv?accessType=DOWNLOAD")
head(nbhds)

`read_csv()` takes the name of a CSV file (or a URL) and produces an R data frame. This is R's answer to a spreadsheet. The commands `head()` and `tail()` give you the first few and the last few lines of the table, respectively. 

Now, the column names of this table are a little ungainly. So we can clean them up a bit. The function `names()` returns the column names of a data frame.

In [ ]:
names(nbhds)

We can now create another "vector" of names that are tidier. The names of the variables are long and some have periods in them. That happens from time to time when you import data from a spread sheet and someone has incuded spaces in the column names. R hates spaces in variable names and so converts them to periods.

The contruction `c(   )` in R is a way to make a vector of data. It "concatenates" the elements in parentheses. We started with 7 names and then replaced them with the vector of our new 7 names.

In [ ]:
names(nbhds) <- c("Borough","FIPS","BNC","Tract","PUMA","NTA","NTA_Name")
head(nbhds)

Loading up our two favorite libraries, we can slice and dice a little. Let's first pull the tracts that are associated with Morningside Heights. Libraries are the way that the R community extends its functionality. In this case one person added a large number of useful tools for working with tables and making great plots. 

In [ ]:
library(dplyr)
library(ggplot2)

Next, we `filter()` on the variable `NTA_Name`, keeping only the rows that refer to Morningside Heights. The `filter()` command is in the `dplyr` package and takes a data frame and a logical expression, keeping only those rows associated with a `TRUE` value. Here the logical operator `==` tests for equality. There's also `<` and `>` for numeric data.

In [ ]:
msh <- filter(nbhds,NTA_Name=="Morningside Heights")

In [ ]:
msh

In the same way, here are the tracts for the Upper West Side.

In [ ]:
uws <- filter(nbhds,NTA_Name=="Upper West Side")

In [ ]:
uws

Alright, if you like, you can create a subset of our data and study just a neighborhood instead of the entire borough, but let's be bold. Recall the number of people who we have answering the census.

In [ ]:
dim(census)

Let's compute a couple of *population quantities* that we'd like to know about the people living here. Add more as you like. We are using the verb `summarize()` from `dplyr` to create summaries from the census. It takes as arguments a data frame and then a number of summary statistics you want to calculate. Here `pop_avg_age` is taking the average or mean age of everyone in Manhattan.

One thing to notice: `ethn=="hisp"` will return 1.6M trues and falses. When we use these values in a mathematical expression, a true value becomes 1 and a false becomes 0. So the average of this expression is the proportion of Manhattanites who are hispanic.

In [ ]:
summarize(census,pop_avg_age=mean(age),pop_prop_his=mean(ethn=="hisp"),pop_prop_female=mean(sex=="female"))

These are *population* figures. **In a typical setting we would never have these.** It is only with a census that we get data on the complete population. Usually, when Politico asks how popular Trump is, or the Behavioral Risk Surveillance Survey (BRFSS) calls to find out about your eating habits, the organization conducts a survey. They would talk to a few hundred or a few hundred people, not 1.6M. 

**Classical confidence intervals**

In our world conducting a survey could be done simply by taking `sample_n()` samples from the `census` data frame. The function `sample_n()` is also from `dplyr`. Here we conduct a single survey of 750 people asking the same questions above -- age, ethnicity and sex.

In [ ]:
n <- 750

survey <- sample_n(census,n)
head(survey)

In [ ]:
dim(survey)

So that gives us a random sample of 750 people and their answers to questions about their age, race, ethnicity, sex and something about where they live. Let's now look at *estimates* of the population quantities derived from our survey, our sample. Same code as above for the population, except now we use `summarize()` to produce summary statistics from `survey` not `census`. Execute the code a few times and notice that each sample involves different people (probably entirely different people) and so our estimates change each time.

In [ ]:
n <- 750

survey <- sample_n(census,n)
summarize(survey,avg_age=mean(age),prop_hisp=mean(ethn=="hisp"),prop_female=mean(sex=="female"))

We see that each time the survey results change a little. Recall that frequentists think about probability arising from repeated events. So to understand the behavior of estimates, they often imagine our single survey as one of a series and talk about the property of that series. 

That will let us infer something about the population information we want to know when we are not in this privileged place of knowing the "truth," but instead all we know is a single survey result. 

So, let's see what happens when we repeat surveys a number of times. Not just once or twice as above, but we will use a *loop* to repeat our survey, in this case 2000 times. Again, no polling agency would ever do this, but it's the kind of thought experiment frequentists use to come up with strategies to learn from data, to learn from a single survey.

A loop is a construction that lets us repeat a computation a number of times. In R, a loop "iterates" over a vector. We will use the function `seq()` to come up with a sequence of numbers. Here's the sequence of 1 to 10. 

In [ ]:
seq(10)

In the code below, we run the survey 2000 times. We create a new dataframe `many_surveys` to hold the 2000 sets of results. It is "initialized" to a NULL or empty object and then we add rows to it as we conduct our surveys.

In [ ]:
# record the average and sd of age in a table (data frame) called many_surveys

many_surveys <- NULL
# repeate it 2000 times, each time adding the mean and sd of age to the many_surveys table

for(i in seq(2000)){
    survey <- sample_n(census,n)
    many_surveys <- bind_rows(many_surveys,
                                summarize(survey,avg_age=mean(age),sd_age=sd(age)))
}

So now we have the results from 2,000 surveys. Each time we get a new sample from the population, we collect their data and we record their ages. We then report the mean and standard deviation of the ages in each survey. So that results in 2,000 averages and sds. 

Here they are.

In [ ]:
head(many_surveys)

Now, let's have a look at how these values change from survey to survey. It seems from the table above that they bounce around a little, but what do they look like? What happens when you repeat the survey a lot of times? What insights do we get?

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)

ggplot() + 
    geom_histogram(aes(x=avg_age),many_surveys,bins=50,color="white",fill="lightblue")+
    xlim(36,42)+
    ggtitle("Sampling Distribution")

The creature above is called the *sampling distribution*. It summarizes what we see when we repeat our survey many times. What does it look like to you? 

Thanks to a lot of clever math, we are looking at something called the Central Limit Theorm at work. It says that averages (in this case, the average age) behaves like a normal distribution if the number of people you talk to is large enough. 

That's great because it means we can use properties of the normal to help with our inference. Below we pull the true population values for the average age in Manhattan and its standard deviation. We add these to our plot. The function `pull()` takes a dataframe and extracts columns -- here we just want age, from which we will compute the mean and standard deviation.

In [ ]:
mu <- mean(pull(census,age))
sigma <- sd(pull(census,age))

ggplot() + 
    geom_histogram(aes(x=avg_age),many_surveys,bins=50,color="white",fill="lightblue") +
    geom_vline(xintercept=mu,color="green")+
    geom_vline(xintercept=mu+2*sigma/sqrt(n),color="green")+
    geom_vline(xintercept=mu-2*sigma/sqrt(n),color="green")+
    xlim(36,42)+
    ggtitle("Sampling Distribution")

It turns out that while the surveys bounce around a lot, their average is essentially right on the true value. So each survey is different from the population value, but it varies in a predictable way. 

In [ ]:
mean(pull(many_surveys,avg_age))
mu

Turns out that the sd's of our surveys are also more or less predictable. Here the average of the sd's from the surveys -- they match the population sd divided by the square root of our survey size. This is an important last ingredient to the Frequentist cabal.

In [ ]:
sd(pull(many_surveys,avg_age))
sigma/sqrt(750)

Now, because we have a normal distribution, we know some things about the way data are distributed around its mean. In this case, we know how the survey results tend to be distributed around the population parameter. 

In [ ]:
normaldata <- data.frame(pts=seq(36,42,len=500),norm=dnorm(seq(36,42,len=500),m=mu,sd=sigma/sqrt(750)))

ggplot() + 
    geom_histogram(aes(x=avg_age,y=..density..),many_surveys,bins=50,color="white",fill="lightblue") +
    geom_line(aes(x=pts,y=norm),normaldata)+
    xlim(36,42)+
    ggtitle("Sampling Distribution + Theory's Predicted Normal Curve")

Here's the trick. Because repeated surveys tend to follow a normal curve centered around the population parameter, we know that 68% of them will be within one standard deviation of the mean and 95% of them will be within two standard deviations of the mean. 

<img src=https://errorstatistics.files.wordpress.com/2014/10/normal-curve-2-3-and-5-sigma.jpg width=500>

And just to check, we get 68% and 95% in our case.

In [ ]:
inside <- nrow(filter(many_surveys, (avg_age < mu+sigma/sqrt(n)) & (avg_age > mu-sigma/sqrt(n))))
inside/2000

In [ ]:
inside <- nrow(filter(many_surveys,(avg_age < mu+2*sigma/sqrt(n)) & (avg_age > mu-2*sigma/sqrt(n))))
inside/2000

OK, so 95% of the survey estimates, mean age, live within two standard deviations of population mean, the number we're after. **We can flip that around and say that 95% of the time, the population mean is within two standard deviations of a survey estimate.** 

So this would give us our 95% confidence interval if only we knew the population standard deviation. Well, using the "plug-in principle", we can just estimate the population sd with our survey sd. That gives us a "classic" frequentist confidence interval for a mean.

We use the `dplyr` function `mutate()` to create new columns from old ones. Here we take `many_surveys` and add a trial number (1 to 2000), and we check whether or not the population mean is within two standard deviation of each row's `avg_age`.

In [ ]:
many_surveys <- mutate(many_surveys,trial=1:2000,cover=mu<avg_age+2*sd_age/sqrt(n) & mu>avg_age-2*sd_age/sqrt(n))
head(many_surveys)

The `many_surveys` data frame now has one row per survey, each reporting the average and sd of people's ages and in the survey. We then form the confidence interval

$$\rm{avg\_age} \pm 2\, \rm{sd\_age}/sqrt(750)$$


The ingredients for the confidence interval are all computable from our survey data. We can now ask how many times is the true population parameter contained in these intervals? Here we take just the first 100.

In [ ]:
ggplot(head(many_surveys,100),aes(x=trial,y=avg_age,color=cover))+
    scale_color_manual("cover", breaks=c(1,2),values=c("red", "lightblue"))+
    geom_errorbar(aes(ymin=avg_age-2*sd_age/sqrt(n), ymax=avg_age+2*sd_age/sqrt(n),color=cover))+
    geom_point(aes(x=trial,y=avg_age,color=cover))+
    geom_hline(yintercept=mu,color="blue")+
    theme(legend.position="none")

If we look at the `cover` variable in our `many_surveys` data frame, we can see what proportion include the true value -- should be close to 95% if everything is workig right.

In [ ]:
summarize(many_surveys,coverage=mean(cover))

**Now, the bootstrap**

The previous setup relied on a fair bit of math to make it go. We had to know the sampling distribution looked essentially normal to be able to flip things around. And we had to know  the formula for the standard deviation of the sampling distribution. **With the bootstrap, we don't have to know any of that.** We are a lot more free because we have a computational mechanism to rely on with proven properties.

For the bootstrap, let's again try to reason just from the sample. Here we take a survey of 750 people.

In [ ]:
n <- 750

survey <- sample_n(census,n)
summarize(survey,avg_age=mean(age),sd_age=sd(age))

Now, the plug-in principle is behind the bootstrap too, but instead of replacing an estimate for the standard deviation, we now "plug in" the sample for the population. This seems like magic and is where the procedure gets its name. We pull ourselves up by our bootstraps. 

To compute the confidence interval for some population parameter from a survey, we repeat the sampling process like we did to get our real samping distribution above. But instead of sampling from the true population which would be expensive, we will clone everyone in our survey 1600000/750 = 2100 times to come up with a kind of synthetic population. By cloning, we again have something we can sample from easily -- we don't have to call to more people. 

Now, if you think about it, because we are cloning each person a lot of times, sampling without replacement from the 1.6M clones is the same as sampling with replacement from our 750 person survey group. And that's how the bootstrap works. Instead of actually repeatedly sampling from the population, we instead sample with replacement from our original survey group. We refer to each sample created this way as a *bootstrap sample*. Here we go.

In [ ]:
boot <- sample_n(survey,n,replace=TRUE)
summarize(boot,avg_age=mean(age))

To approximate the sampling distribution for our estimate of the mean age in the population, we redo this many times. Here I take 5,000. It is usually a big-ish number. But since we re just resampling from the survey, it's fast.

In [ ]:
# we'll store the summary statistics in boots

boots <- NULL

# repeat this 5000 times, building up boots as we go

for(i in seq(5000)){
    boot <- sample_n(survey,n,replace=TRUE)
    boots <- bind_rows(boots,summarize(boot,avg_age=mean(age)))
}

This then gives us an approximation to the sampling distribution. 

In [ ]:
ggplot() + 
    geom_histogram(aes(x=avg_age,y=..density..),boots,bins=50,color="white",fill="lightblue")

We form our confidence interval directly from these data, again, all derived from the original survey data.

In [ ]:
boot_avg <- mean(pull(boots,avg_age))
boot_sd <- sd(pull(boots,avg_age))

In [ ]:
boot_avg-2*boot_sd
boot_avg+2*boot_sd
mu

Notice that the truth is inside the confidence interval. We could have also used the 0.95 and 0.025 quantiles if we didn't think the approximation looked normal.

In [ ]:
quantile(pull(boots,avg_age),c(0.025,0.975))

To show this works, we will repeat the process 25 times and see what the coverage looks like.

In [ ]:
n <- 750

many_boots <- NULL

for(i in seq(25)){
        
    boots <- NULL

    # 1000 bootstrap samples just to speed up computation
    for(j in seq(1000)){
        boot <- sample_n(survey,n,replace=TRUE)
        boots <- bind_rows(boots,summarize(boot,avg_age=mean(age)))
    }
    
    many_boots <- bind_rows(many_boots,summarize(boots,boot_avg=mean(avg_age),boot_sd=sd(avg_age)))
}

We will again ask how many times is the mean contained in the interval and create the plot from above. 

In [ ]:
many_boots <- mutate(many_boots,trial=1:25,cover=mu<boot_avg+2*boot_sd & mu>boot_avg-2*boot_sd)

In [ ]:
many_boots

And our plot...

In [ ]:
ggplot(head(many_boots,100),aes(x=trial,y=boot_avg,color=cover))+
    scale_color_manual("cover", breaks=c(1,2),values=c("red", "lightblue"))+
    geom_errorbar(aes(ymin=boot_avg-2*boot_sd, ymax=boot_avg+2*boot_sd,color=cover))+
    geom_point(aes(x=trial,y=boot_avg,color=cover))+
    geom_hline(yintercept=mu,color="blue")+
    theme(legend.position="none")

Now, suppose we want another statistic, say the median age. 

In [ ]:
n <- 750 # our survey size

# we'll store the summary statistics in boots
boots <- NULL

# repeat this 5000 times, building up boots as we go

for(i in seq(5000)){
    boot <- sample_n(survey,n,replace=TRUE)
    boots <- bind_rows(boots,summarize(boot,med_age=median(age)))
}

In [ ]:
summarize(boots,lower=quantile(med_age,0.025),upper=quantile(med_age,0.975))

In [ ]:
median(pull(census,age))